# 0.0. Imports

In [2]:
import pandas as pd
import inflection
import math
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

from IPython.core.display import HTML
from IPython.display import Image

## 0.1. Helper Functions

In [3]:
def jupyter_settings():
     %matplotlib inline
     %pylab inline
     plt.style.use( 'bmh' )
     plt.rcParams['figure.figsize'] = [25, 12]
     plt.rcParams['font.size'] = 24
     display( HTML( '<style>.container { width:100% !important; }</style>') )
     pd.options.display.max_columns = None
     pd.options.display.max_rows = None
     pd.set_option( 'display.expand_frame_repr', False )
     sns.set()
 
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

In [4]:
df_sales_raw = pd.read_csv('datasets/train.csv', low_memory=False)
df_store_raw = pd.read_csv('datasets/store.csv', low_memory=False)

#merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store' )

In [5]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
570269,175,3,2014-02-05,6151,683,1,1,0,0,c,a,4130.0,NaN,NaN,0,NaN,NaN,NaN


# 1.0. Descrição dos Dados

In [6]:
#copy of the dataset to recover the original dataset
df1 = df_raw.copy()

## 1.1. Rename Columns
The idea here is to get agility on development through easy names on the columns

In [7]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2. Data Dimensions

In [8]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 1017209
Number of cols: 18


## 1.3. Data Types

In [9]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4. Check NA

In [10]:
# return how many NA's contains on each column
df1.isna().sum() 

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5. Fillout NA

In [ ]:
# competition_distance      
# Here i'm fillin the NA's with a value that is much higher than the max value for competitor distance on the dataset
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000 if math.isnan(x) else x)

# competition_open_since_month
# Here, I'm assuming that is important to have this information filled (M02_V02_9min)
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month'])
                                                                                else x['competition_open_since_month'], axis=1)

# competition_open_since_year 
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year'])
                                                                                else x['competition_open_since_year'], axis=1)
                 
# promo2_since_week      

df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year    
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

In [ ]:
month_map = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

# fill na's with 0 to avoid the comparison using 'isnan'
df1['promo_interval'].fillna(0,inplace=True)

# extract the month of the 'date' column and apply the dictionary created above to use as future comparison.
df1['month_map'] = df1['date'].dt.month.map(month_map)

# verifying if the store is participating in the promo, based on column 'date', represented by 'month_map'
df1['is_promo'] = df1[['promo_interval','month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 
                                                            else 1 if x['month_map'] in x['promo_interval'].split(',')
                                                            else 0, axis=1)

In [ ]:
# observer that we don't have any columns with NA anymore
df1.isna().sum()

## 1.6. Change Types
It's important to verify if the types are correct after many modifications on the variables

In [ ]:
# These variables were float64.
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype('int64')
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype('int64')
df1['promo2_since_week'] = df1['promo2_since_week'].astype('int64')
df1['promo2_since_year'] = df1['promo2_since_year'].astype('int64')

## 1.7. Descriptive Statistics
This topic is important to:
- Understand more from the business
- Identify errors

In [ ]:
#allocating the numerical columns on a new variable
num_attributes = df1.select_dtypes(include = ['int64','float64'])

# allocating the categorical columns on a new variable
cat_attributes = df1.select_dtypes(exclude = ['int64','float64','datetime64[ns]'])

### 1.7.1. Numerical Attributes

In [ ]:
# Central Tendency - Mean, Median
ct1 = pd.DataFrame(num_attributes.apply(np.mean) ).T # mean
ct2 = pd.DataFrame(num_attributes.apply(np.median) ).T # median

# Dispersion - std, min, max, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std) ).T # standard deviation
d2 = pd.DataFrame(num_attributes.apply(min) ).T # min
d3 = pd.DataFrame(num_attributes.apply(max) ).T # max
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min()) ).T # range
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T # skew
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T # kurtosis

# concatenate
m = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns = ['Attributes', 'Min', 'Max', 'Range', 'Mean', 'Median', 'Std', 'Skew', 'Kurtosis']

m

In [ ]:
sns.distplot(df1['competition_distance'])

### 1.7.2. Categorical Attributes

In [ ]:
# Visualize how many attributes each column has
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales']>0)]
plt.subplot(1,3,1)
sns.boxplot(x='state_holiday' ,y='sales', data=aux1 )

plt.subplot(1,3,2)
sns.boxplot(x='store_type' ,y='sales', data=aux1 )

plt.subplot(1,3,3)
sns.boxplot(x='assortment' ,y='sales', data=aux1 )

# 2.0. Feature Engineering

## 2.1. Mind Map Hypothesis

In [ ]:
df2 = df1.copy()

In [ ]:
Image('img/MindMapRossman2.png')

## 2.2. Hypothesis Creation

### 2.2.1. Hipóteses da Loja

**1.** Lojas com maior quadro de funcionários deveriam vender mais.

**2.** Lojas com maior capacidade de estoque deveriam vender mais.

**3.** Lojas com maior porte deveriam vender mais.

**4.** Lojas com competidores mais próximos deveriam vender menos

**5.** Lojas com maior sortimento deveriam vender mais.

**6.** Lojas com competidores à mais tempo deveriam vender mais

#### 2.2.2. Hipóteses do Produto

**1.** Lojas que investem mais em marketing deveriam vender mais.

**2.** Lojas que expõe mais o produto nas vitrines deveriam vender mais.

**3.** Lojas que tem preços menores deveriam vender mais.

**4.** Lojas com promoções ativas por mais tempo deveriam vender mais.

**5.** Lojas com mais dias de promoção deveriam vender mais.

**6.** Lojas com promoções mais agressivas deveriam vender mais.

**7.** Lojas com mais promoções consecutivas deveriam vender mais.

**8.** Lojas com promoções mais agressivas (maiores descontos) deveriam vender mais

### 2.2.3. Hipóteses do tempo

**1.** Lojas deveriam vender mais no segundo semestre do ano

**2.** Lojas deveriam vender menos aos finais de semana.

**3.**Lojas abertas durante o feriado de Natal deveriam vender mais.

**4.**Lojas deveriam vender mais ao longo dos anos

**5.**Lojas deveriam vender mais depois do dia 10 de cada mês

**6.**Lojas deveriam vender menos durante feriados escolares.

## 2.3. lista Final de Hipóteses
Etapa de priorização de hipóteses. A ideia aqui é simples: Priorizar as hipóteses que já possuem os dados à disposição e tratados.

**5.** Lojas com maior sortimento deveriam vender mais.

**4.** Lojas com competidores mais próximos deveriam vender menos

**6.** Lojas com competidores à mais tempo deveriam vender mais

**4.** Lojas com promoções ativas por mais tempo deveriam vender mais.

**5.** Lojas com mais dias de promoção deveriam vender mais.

**7.** Lojas com mais promoções consecutivas deveriam vender mais.

**1.** Lojas deveriam vender mais no segundo semestre do ano

**2.** Lojas deveriam vender menos aos finais de semana.

**3.** Lojas abertas durante o feriado de Natal deveriam vender mais.

**4.** Lojas deveriam vender mais ao longo dos anos

**5.** Lojas deveriam vender mais depois do dia 10 de cada mês

**6.** Lojas deveriam vender menos durante feriados escolares.

## 2.4. Feature Engineering

In [ ]:
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

In [ ]:
# competition since
# gather 'competition_open_since_year' and 'competition_open_since_month' together and then
# subtracting it to 'date' so we can obtain how many months have passed for each store since
# competitions opened
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year=x['competition_open_since_year'],
                  month=x['competition_open_since_month'],
                 day=1), axis=1)

# dividing by 30 so we can obtain the result as months
df2['competition_time_month'] = ( (df2['date'] - df2['competition_since'])/30 ).apply(lambda x: x.days).astype(int)

In [ ]:
# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)

# now we have to convert the 'promo_since' to datetime. This method is explained
# on the bonus video, that is not launched at the moment (26/03) 
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w' ) - datetime.timedelta(days = 7) )
df2['promo_time_week'] = ( (df2['date'] - df2['promo_since'] )/7 ).apply(lambda x: x.days ).astype(int)

In [ ]:
# assortment
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b'
                                          else 'extended')

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' 
      else 'christimas' if x == 'c' else 'regular_day')

In [ ]:
df2.head().T

# 3.0. Filtragem de Variáveis

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

## 3.1. Filtragem das Linhas

In [ ]:
# stores open only
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2. Seleção das colunas

In [ ]:
#'customers' é uma restrição do negócio, pois não teremos o input no momento da análise de quantas pessoas
# estarão nas lojas nas próximas 6 semanas
# a coluna 'open' não será mais necessária pois estará preenchida totalmente apenas com o valor '1'
# 'promo_interval' e 'month_map' são variáveis auxiliares, e não mais necessárias
cols_drop = ['customers','open','promo_interval','month_map']
df3 = df3.drop(cols_drop, axis=1)